# Cartographic Rendering with GeoVista

This example shows how to combine pyvista-xarray with [GeoVista](https://github.com/bjlittle/geovista) for cartographic 3D rendering. GeoVista transforms lat/lon data onto a 3D globe with coastlines and base layers.

We use NOAA's Optimum Interpolation SST (OISST) dataset to visualize sea surface temperature estimation errors on a 3D globe with a Blue Marble texture.

In [ ]:
import geovista as gv
import xarray as xr

from pvxarray.vtk_source import PyVistaXarraySource

## Load OISST Data

The OISST dataset provides global SST on a regular lat/lon grid. We create a `PyVistaXarraySource` so GeoVista can consume the data through VTK's pipeline:

In [ ]:
# From https://www.ncei.noaa.gov/products/optimum-interpolation-sst
ds = xr.open_dataset("data/oisst-avhrr-v02r01.19810901.nc")
da = ds.err

source = PyVistaXarraySource(
    da,
    x="lon",
    y="lat",
    z="zlev",
    time="time",
    resolution=1.0,
)

## Render on a 3D Globe

GeoVista's `GeoPlotter` projects the rectilinear data onto a sphere. We add coastlines for geographic reference and use depth peeling for correct rendering of transparent regions (where SST data is missing over land):

In [ ]:
plotter = gv.GeoPlotter()
plotter.add_mesh(source, cmap="coolwarm", nan_opacity=0)
plotter.add_base_layer(texture=gv.blue_marble())
plotter.add_coastlines(resolution="10m", color="white")
plotter.enable_depth_peeling()
plotter.view_isometric()
plotter.show()